In [1]:
from numpy import mean
from numpy import std
from numpy import dstack
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.regularizers import l2
from tensorflow.keras.metrics import *
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.experimental.set_visible_devices(gpus[2], 'GPU')
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

4 Physical GPUs, 1 Logical GPU


In [3]:
s25 = np.load('s25.npy')
h25 = np.load('h25.npy')

In [4]:
s25_labels = np.array([0 for _ in range(0,len(s25))])
h25_labels = np.array([1 for _ in range(0,len(h25))])

In [5]:
X = np.append(s25,h25,axis=0)

In [6]:
y = np.append(s25_labels,h25_labels,axis=0)

In [7]:
X.shape

(1142, 6250, 19)

In [8]:
def evaluate_model(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 10, 32
    n_timesteps, n_features = trainX.shape[1], trainX.shape[2]
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features),kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu',kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
    model.add(Dropout(0.5))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(100, activation= 'relu',kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01) ))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid',kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01) ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy',Precision(),Recall(),AUC()])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy,precision,recall,auc = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy,precision,recall,auc

In [9]:
def summarize_results(accuracies,precisions,recalls,aucs):
    m, s = mean(accuracies), std(accuracies)
    print( ' Accuracy: %.3f%% (+/-%.3f) ' % (m, s))
    m, s = mean(precisions), std(precisions)
    print( ' Precision: %.3f%% (+/-%.3f) ' % (m, s))
    m, s = mean(recalls), std(recalls)
    print( ' Recall: %.3f%% (+/-%.3f) ' % (m, s))
    m, s = mean(aucs), std(aucs)
    print( ' AUC: %.3f%% (+/-%.3f) ' % (m, s))

In [10]:
def run_experiment(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    mean = trainX.mean(axis=0)
    trainX -= mean
    std = trainX.std(axis=0)
    trainX /= std
    
    testX -= mean
    testX /= std
    
    
    # repeat experiment
    accuracies = list()
    precisions = list()
    recalls = list()
    aucs = list()
    for r in range(repeats):
        accuracy,precision,recall,auc = evaluate_model(trainX, trainy, testX, testy)
        accuracy = accuracy * 100.0
        precision = precision * 100.0
        recall = recall * 100.0
        auc = auc * 100.0
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        aucs.append(auc)
    # summarize results
    summarize_results(accuracies,precisions,recalls,aucs)

In [11]:
run_experiment(X,y)

Epoch 1/10
29/29 [==============================] - 1s 34ms/step - loss: 7.3592 - accuracy: 0.6101 - precision: 0.5874 - recall: 0.5654 - auc: 0.6215
Epoch 2/10
29/29 [==============================] - 1s 33ms/step - loss: 3.5058 - accuracy: 0.8280 - precision: 0.8379 - recall: 0.7850 - auc: 0.9218
Epoch 3/10
29/29 [==============================] - 1s 27ms/step - loss: 2.3374 - accuracy: 0.9113 - precision: 0.8699 - recall: 0.9533 - auc: 0.9705
Epoch 4/10
29/29 [==============================] - 1s 27ms/step - loss: 1.7676 - accuracy: 0.9507 - precision: 0.9382 - recall: 0.9579 - auc: 0.9860
Epoch 5/10
29/29 [==============================] - 1s 25ms/step - loss: 1.4296 - accuracy: 0.9770 - precision: 0.9722 - recall: 0.9790 - auc: 0.9964: 0s - loss: 1.4567 - accuracy: 0.9759 - precision: 0.9725 - recall: 0.9755 - auc: 0.9
Epoch 6/10
29/29 [==============================] - 1s 25ms/step - loss: 1.2562 - accuracy: 0.9660 - precision: 0.9606 - recall: 0.9673 - auc: 0.9924
Epoch 7/10
29/

In [12]:
def evaluate_model_2(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 10, 32
    n_timesteps, n_features = trainX.shape[1], trainX.shape[2]
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01),activation='relu', input_shape=(n_timesteps,n_features)))
    model.add(Dropout(0.5))
    model.add(Conv1D(filters=64, kernel_size=3, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01),activation='relu'))
    model.add(Dropout(0.5))
    model.add(Conv1D(filters=64, kernel_size=3, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01),activation='relu'))
    model.add(Dropout(0.5))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(100, activation= 'relu',kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01) ))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid',kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01) ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy',Precision(),Recall(),AUC()])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy,precision,recall,auc = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy,precision,recall,auc

In [13]:
def run_experiment_2(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    mean = trainX.mean(axis=0)
    trainX -= mean
    std = trainX.std(axis=0)
    trainX /= std
    
    testX -= mean
    testX /= std
    # repeat experiment
    accuracies = list()
    precisions = list()
    recalls = list()
    aucs = list()
    for r in range(repeats):
        accuracy,precision,recall,auc = evaluate_model_2(trainX, trainy, testX, testy)
        accuracy = accuracy * 100.0
        precision = precision * 100.0
        recall = recall * 100.0
        auc = auc * 100.0
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        aucs.append(auc)
    # summarize results
    summarize_results(accuracies,precisions,recalls,aucs)

In [14]:
run_experiment_2(X,y)

Epoch 1/10
29/29 [==============================] - 1s 35ms/step - loss: 10.2471 - accuracy: 0.5235 - precision_5: 0.4891 - recall_5: 0.3668 - auc_5: 0.5582
Epoch 2/10
29/29 [==============================] - 1s 33ms/step - loss: 4.9263 - accuracy: 0.5641 - precision_5: 0.5630 - recall_5: 0.3131 - auc_5: 0.6613
Epoch 3/10
29/29 [==============================] - 1s 32ms/step - loss: 3.6363 - accuracy: 0.7021 - precision_5: 0.7500 - recall_5: 0.5467 - auc_5: 0.8220
Epoch 4/10
29/29 [==============================] - 1s 35ms/step - loss: 2.8595 - accuracy: 0.8094 - precision_5: 0.7848 - recall_5: 0.8178 - auc_5: 0.9038
Epoch 5/10
29/29 [==============================] - 1s 27ms/step - loss: 2.3359 - accuracy: 0.8959 - precision_5: 0.8758 - recall_5: 0.9065 - auc_5: 0.9589
Epoch 6/10
29/29 [==============================] - 1s 31ms/step - loss: 2.0281 - accuracy: 0.9135 - precision_5: 0.8921 - recall_5: 0.9276 - auc_5: 0.9724
Epoch 7/10
29/29 [==============================] - 1s 28ms/ste

In [15]:
def evaluate_model_3(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 10, 32
    n_timesteps, n_features = trainX.shape[1], trainX.shape[2]
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), input_shape=(n_timesteps,n_features)))
    model.add(Conv1D(filters=64, kernel_size=3, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), activation='relu'))
    model.add(Dropout(0.5))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(100, activation= 'relu',kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01) ))
    model.add(Dropout(0.25))
    model.add(Dense(50, activation= 'relu',kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01) ))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid',kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01) ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy',Precision(),Recall(),AUC()])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy,precision,recall,auc = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy,precision,recall,auc

In [16]:
def run_experiment_3(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    mean = trainX.mean(axis=0)
    trainX -= mean
    std = trainX.std(axis=0)
    trainX /= std
    
    testX -= mean
    testX /= std
    # repeat experiment
    accuracies = list()
    precisions = list()
    recalls = list()
    aucs = list()
    for r in range(repeats):
        accuracy,precision,recall,auc = evaluate_model_3(trainX, trainy, testX, testy)
        accuracy = accuracy * 100.0
        precision = precision * 100.0
        recall = recall * 100.0
        auc = auc * 100.0
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        aucs.append(auc)
    # summarize results
    summarize_results(accuracies,precisions,recalls,aucs)

In [17]:
run_experiment_3(X,y)

Epoch 1/10
29/29 [==============================] - 1s 24ms/step - loss: 9.4345 - accuracy: 0.5301 - precision_10: 0.4984 - recall_10: 0.3668 - auc_10: 0.5146
Epoch 2/10
29/29 [==============================] - 1s 31ms/step - loss: 4.1679 - accuracy: 0.6287 - precision_10: 0.7171 - recall_10: 0.3435 - auc_10: 0.7541
Epoch 3/10
29/29 [==============================] - 1s 23ms/step - loss: 3.0541 - accuracy: 0.6090 - precision_10: 0.7934 - recall_10: 0.2243 - auc_10: 0.8196
Epoch 4/10
29/29 [==============================] - 1s 25ms/step - loss: 2.4292 - accuracy: 0.8554 - precision_10: 0.8318 - recall_10: 0.8668 - auc_10: 0.9113
Epoch 5/10
29/29 [==============================] - 1s 28ms/step - loss: 2.0610 - accuracy: 0.9277 - precision_10: 0.9269 - recall_10: 0.9182 - auc_10: 0.9777
Epoch 6/10
29/29 [==============================] - 1s 36ms/step - loss: 1.7086 - accuracy: 0.9682 - precision_10: 0.9650 - recall_10: 0.9673 - auc_10: 0.9944
Epoch 7/10
29/29 [============================